In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'biznews_com'
c = Crawl()  # creating object
cl_data = dashboard['core_web_india']
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_rss_page = 0
publish_source = 'biznews.com'
country = 'India'
language = 'English'
images_path = []


rss_pages = c.download_page('https://biznews.fiu.edu/feed/')

for i in rss_pages.split('<item>')[1:]:

        # source_link
        source_link = c.scrap('<link>(.*?)</link>', i)
        
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue      
            
        time.sleep(random.randint(1,3))
        
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_url += 1
                continue  
       
        # source_headline
        source_headline = c.scrap('<title>(.*?)</title>', page)
     
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

        logger.info(f'Fetching {source_link}\n')
        
          # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<pubDate>\w+,\s+(.*?) \+', i)
            date_time_obj = datetime.strptime(date_time_str, '%d %b %Y %H:%M:%S')
            # There is no time difference between India and Sri Lanka
            ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue
            
        # break if date is not today's date
#         if pub_date != created_on:
#             break  
            
         # journalist   
        journalist = c.scrap('<span\s*class="author\s*vcard">(.*?)</span>', page)
        journalist = c.strip_html(journalist)
        if not journalist: journalist = 'NA'
        
         # source_content          
        source_content = c.scrap('<div\s*class="entry-content .*?>(.*?)</span>', page)
        source_content = c.strip_html(source_content)


         # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1

logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_page}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')            

https://biznews.fiu.edu/2021/03/fiu-business-international-mba-program-ranked-no-3-in-nation-by-u-s-news/
FIU Business International MBA program ranked No. 3 in nation by U.S. News &#8211; BizNews
FIU Business
U.S. News &amp; World Report has ranked the FIU Business International MBA Program at No. 3 in the U.S., a jump from No. 8 last year, and the highest graduate program ranking in the college’s history. The new ranking places FIU Business right behind Harvard Business School and ahead of business schools at such institutions as the University of Pennsylvania, Georgetown University, Columbia University and New York University, all of which are also in the Top 10.
“We are honored to see our International MBA program ranked among some of the world’s most elite programs,” said Joanne Li, dean of FIU Business. “The ranking reflects our continued commitment to excellence in international business education and research, our leadership in this area, and our success in impacting our gradua

Altria grant establishes FIU Business program to promote diversity and inclusion. &#8211; BizNews
Ellen Forman
As part of its ongoing effort to combat structural racism, FIU Business has been awarded a $100,000 grant from Altria to establish a unique multi-platform program designed to train more inclusive leaders and further the goals of diversity and justice.
The three-part project strives to open a conversation and programmatically address challenges of inclusion and equity through financial support, academic coursework and purpose-driven dialogue.
The grant, running through summer 2021, will fund:

The Altria “Real Talk” Series, bringing together Altria team members and FIU Business students for a series of conversations on structural racism, with the goal of developing a more just, inclusive workforce,
A pilot program for development and implementation of a diversity micro-credential by FIU Business faculty members over a one year period, with the potential for expansion to alumni 

FIU Business receives $1.7 million grant to expand financial literacy program. &#8211; BizNews
Cynthia Corzo
FIU Business has received a $1.7 million grant from the Truist Foundation to expand the reach of a high school financial literacy program, delivered through the Truist FIU Financial Wellness Clinic, to school systems across the U.S.
Currently, students of four public high schools in Miami-Dade County and parents participate in the National Academy Foundation’s (NAF) Academy of Finance (AOF) program. The five-year grant will allow the clinic team to deliver the Truist-FIU Finance curriculum to eight high schools in Miami-Dade County Public Schools.
Additionally, the program, which began in person and is now offered virtually, will be made available to schools in other Florida districts and nationally. On the national level, teachers and high school students in at least 140 U.S. cities will have access to the curriculum.
The technology-intensive Truist FIU Financial Wellness Clini